In [1]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-04 21:54:19--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  73.9MB/s    in 0.3s    

2021-05-04 21:54:20 (73.9 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/cats/cat

In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [5]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_img_sizes = []
cats_img_sizes = []
dogs_img_sizes = []
horses_img_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_img_sizes.append(hu.shape[:-1])
  cats_img_sizes.append(c.shape[:-1])
  dogs_img_sizes.append(d.shape[:-1])
  horses_img_sizes.append(ho.shape[:-1])


In [6]:
# print(np.mean(human_img_sizes, axis=0))
# print(np.mean(cats_img_sizes, axis=0))
# print(np.mean(dogs_img_sizes, axis=0))
# print(np.mean(horses_img_sizes, axis=0))

mean_sizes = [np.mean(human_img_sizes, axis=0), np.mean(cats_img_sizes, axis=0),
              np.mean(dogs_img_sizes, axis=0), np.mean(horses_img_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [7]:
image_gen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [8]:
train = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_size = [304,329,3]

In [10]:
input = Input(shape = img_size)
model = ResNet50(input_tensor=input,)

102973440/102967424 [==============================] - 1s 0us/step


In [11]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [12]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [14]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [15]:
model.fit(train, validation_data=val, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 56s 123ms/step - loss: 1.5993 - accuracy: 0.2528 - val_loss: 1.3559 - val_accuracy: 0.3500
Epoch 2/50
182/182 [==============================] - 21s 117ms/step - loss: 1.4100 - accuracy: 0.2639 - val_loss: 1.3792 - val_accuracy: 0.3500
Epoch 3/50
182/182 [==============================] - 21s 116ms/step - loss: 1.3017 - accuracy: 0.3748 - val_loss: 1.2603 - val_accuracy: 0.4250
Epoch 4/50
182/182 [==============================] - 21s 117ms/step - loss: 1.3019 - accuracy: 0.3754 - val_loss: 1.3157 - val_accuracy: 0.3375
Epoch 5/50
182/182 [==============================] - 21s 116ms/step - loss: 1.3104 - accuracy: 0.3834 - val_loss: 1.3235 - val_accuracy: 0.3250
Epoch 6/50
182/182 [==============================] - 21s 116ms/step - loss: 1.2933 - accuracy: 0.3820 - val_loss: 1.2558 - val_accuracy: 0.4125
Epoch 7/50
182/182 [==============================] - 21s 118ms/step - loss: 1.2445 - accuracy: 0.4385 - val_loss: 1.2219 - val_ac

In [16]:
model.history.history

{'accuracy': [0.28434064984321594,
  0.30906593799591064,
  0.375,
  0.3997252881526947,
  0.36950549483299255,
  0.4217033088207245,
  0.4326923191547394,
  0.4491758346557617,
  0.42032966017723083,
  0.45879119634628296,
  0.46291208267211914,
  0.424450546503067,
  0.46978020668029785,
  0.4752747118473053,
  0.46978020668029785,
  0.4670329689979553,
  0.46291208267211914,
  0.48351648449897766,
  0.4972527325153351,
  0.4931318759918213,
  0.5247252583503723,
  0.4821428656578064,
  0.5109890103340149,
  0.526098906993866,
  0.4972527325153351,
  0.5233516693115234,
  0.5398351550102234,
  0.5123626589775085,
  0.5398351550102234,
  0.5370879173278809],
 'loss': [1.4805240631103516,
  1.3933517932891846,
  1.310252070426941,
  1.293307900428772,
  1.3127621412277222,
  1.2558436393737793,
  1.2353856563568115,
  1.2160462141036987,
  1.2075482606887817,
  1.2130094766616821,
  1.2043906450271606,
  1.2215814590454102,
  1.1698065996170044,
  1.1600335836410522,
  1.15655469894409